本示例有以下几个功能  
1.作为仿真验证脚本，`generate_MVM`函数随机生成用于仿真测试的输入、权重和bias矩阵（both FP and FXP）。  
>其中FP用于软件仿真，输出软件仿真结果。FXP用于硬件读取。最后TB会自动比对软硬件的结果，完成验证。  

2.`generate_MVM`底层依赖于generate_Q,而generate_Q函数即可以随机生成，也可以从外部读取numpy。
>如果是从外部读取numpy，那就可以做成实测

In [1]:
from mvm_verify_utils import *

#定义dense_layer每层的输入输出维度
DENSE_LAYER_CONFIG = [
    (32,100),     
    (100,128),
    (128,8)
]
FXP_CFG ={
    'din':(9,7),
    'weight':(1,7),
    'bias':(1,14)
}
#定义是仿真还是实测
this_test_condition =  TEST_CONDITION.sim

if(this_test_condition == TEST_CONDITION.sim):
    generate_random_layer_param(DENSE_LAYER_CONFIG,seed=0)

#读取待测试数据并转成FXP
test_data_dict=npy2fxp(DENSE_LAYER_CONFIG,FXP_CFG)

save_test_dict2txt(test_data_dict,DENSE_LAYER_CONFIG,FXP_CFG)

verify_dense_layer(test_data_dict,DENSE_LAYER_CONFIG,2,0)

mem address info: [('0', '0'), ('80', '4'), ('210', '8')]
 D E B U G    I N F O   (layer=2,pe=0)
------------------------------------------------------------
cnt         din                            * weight     = result          
------------------------------------------------------------
0         + 0.5465109215840517             * 0.015625   = 0.008539233149750807 
1         + -0.2778600738608944            * -0.0625    = 0.02590548776605671 
2         + -0.5706373001344789            * -0.0546875 = 0.057112215117161025 
3         + -0.7434020042290317            * 0.2265625  = -0.11131480146597897 
4         + -0.8097753954028756            * -0.1015625 = -0.02907198787037442 
5         + -0.3248504166662288            * -0.0390625 = -0.016382518469349856 
6         + -0.9725679788629001            * 0.625      = -0.6242375052586625 
7         + -0.03826261130096177           * 0.046875   = -0.6260310651633951 
8         + -0.7095450510695397            * -0.109375  = -0.5484245

In [1]:
import numpy as np
from fixedpoint import FixedPoint

def WeightAndBias_Reshape(matrix,WorB='weight'):
    FillZero = '0'*8 if WorB=='weight' else '0'*15
    shape_row,shape_column=matrix.shape
    piece_num     = shape_column //32
    column_remain = shape_column % 32
    if(piece_num==0):
        pass
    else:
        result = matrix[:,0:32]
        for i in range(1,piece_num):
            piece  = matrix[:,i*32:(i+1)*32]
            result = np.concatenate((result, piece), axis=0)
    if(column_remain==0):
        pass
    else:
        i=i+1
        piece  = np.concatenate((matrix[:,i*32:],np.array(['00000000' for _ in range(shape_row*(32-column_remain))]).reshape(shape_row,(32-column_remain))),axis=1)
        result = np.concatenate((result, piece), axis=0)
    # result_tmp0 = np.flip(result,axis=1)
    result_tmp0 = result[:,::-1]
    result_tmp1 = [''.join(row) for row in  result_tmp0]
    return result_tmp1
    
def generate_Q(mean,size,thresh_abs,std_dev,m,n):
    # 生成正态分布数据
    data = np.random.normal(mean, std_dev, size)
    data = np.clip(data, -thresh_abs, thresh_abs)
    data_fxp=[FixedPoint(item, signed=True, m=m, n=n, str_base=2, overflow='clamp', overflow_alert='ignore') for item in data]
    return ([float(item) for item in data_fxp],[str(item) for item in data_fxp])

def generate_MVM(dimi,dimo,seed,debug_vv_col = -1):
    np.random.seed(seed)
    din    = generate_Q(0,dimi,2,0.5,9,7)
    weight = generate_Q(0,dimi*dimo,1,0.2,1,7)
    bias   = generate_Q(0,dimo,1,0.2,1,14) 
    
    din_fp    = np.array(din[0]).reshape(1,dimi)
    weight_fp = np.array(weight[0]).reshape(dimi,dimo)
    bias_fp   = np.array(bias[0]).reshape(1,dimo)

    weight_str = np.array(weight[1]).reshape(dimi,dimo)
    bias_str   = np.array(bias[1]).reshape(1,dimo)
    
    with open('../output/tmp/din.txt', 'w') as f_din, open('../output/tmp/weight.txt', 'w') as f_weight, open('../output/tmp/bias.txt', 'w') as f_bias:
        f_din.write('\n'.join(din[1]))  # Write din strings to din.txt
        f_weight.write('\n'.join(WeightAndBias_Reshape(weight_str,'weight')))  # Write weight strings to weight.txt
        f_bias.write('\n'.join(WeightAndBias_Reshape(bias_str,'bias')))  # Write bias strings to bias.txt
        
    z = np.dot(din_fp,weight_fp) + bias_fp
    output = np.tanh(z)
    output = output.astype(str).reshape(-1).tolist()


        
    with open('../output/tmp/gold_result.txt','w') as f:
        f.write('\n'.join(output))


    if(debug_vv_col>=0):
        print(f"\33[1;7;32m D E B U G    I N F O   ({debug_vv_col})\33[0m")
        print('-'*60)
        print(f"  {'din':<10} * {'weight':<10} = {'result':<15} ")
        print('-'*60)
        din_vec     = din_fp[0]
        weight_vec  = weight_fp[:,debug_vv_col] 
        bias_scalar = bias_fp[0][debug_vv_col]
        mac_result  = 0
        for i in range(len(din_vec)):
            mac_result += din_vec[i]*weight_vec[i]
            print(f"+ {din_vec[i]:<10} * {weight_vec[i]:<10} = {mac_result:<15} ")
        mac_result += bias_scalar
        dbg_result  = np.tanh(mac_result)
        print('-'*60)
        print(f"  {'bias':<18} = {'result':<15} ")
        print('-'*60)
        print(f"+ {bias_scalar:<18} = {mac_result:<15}")
        print('-'*60)
        print(f"{'tanh_result':<18}")
        print('-'*60)
        print(f"{dbg_result:<18}")
        print()
    return output
output = generate_MVM(32,100,1024,debug_vv_col=32)
print(output)


 D E B U G    I N F O   (32)
------------------------------------------------------------
  din        * weight     = result          
------------------------------------------------------------
+ 1.0625     * -0.03125   = -0.033203125    
+ 0.125      * -0.140625  = -0.05078125     
+ 0.7265625  * -0.1796875 = -0.18133544921875 
+ 0.28125    * -0.015625  = -0.18572998046875 
+ 0.2265625  * 0.15625    = -0.15032958984375 
+ -0.40625   * 0.203125   = -0.23284912109375 
+ 0.4296875  * -0.0546875 = -0.25634765625  
+ 0.1015625  * -0.0625    = -0.2626953125   
+ -0.9375    * -0.0078125 = -0.25537109375  
+ -0.28125   * 0.078125   = -0.27734375     
+ -0.03125   * -0.21875   = -0.2705078125   
+ 0.40625    * -0.265625  = -0.37841796875  
+ -0.2890625 * 0.0859375  = -0.40325927734375 
+ 0.2890625  * -0.1328125 = -0.441650390625 
+ -0.171875  * 0.265625   = -0.4873046875   
+ 0.1484375  * 0.375      = -0.431640625    
+ -0.1875    * -0.2578125 = -0.38330078125  
+ 0.078125   * 0.2421875  = -